In [ ]:
import requests
import json
import pandas as pd
import os
import dotenv
import time
import base64
dotenv.load_dotenv()
FDAPI = os.environ["FUND_DAILY_API_KEY"]
FFAPI = os.environ["FUND_FACT_API_KEY"]
FundDailyKey = {
    'Ocp-Apim-Subscription-Key': FDAPI
}
FundFactKey = {
    'Ocp-Apim-Subscription-Key': FFAPI
}

In [ ]:
project_type_keys = ['proj_retail_type', 'proj_term_flag', 'proj_term_dd', 'proj_term_mm','proj_term_yy']
specification_keys = ['spec_code', 'spec_desc']
suitability_keys = ['risk_spectrum', 'risk_spectrum_desc','fund_suitable_desc']

In [ ]:
def FundFactAPIGet(url, proj_unique_id, type, key):
    if type == 'amc':
        response = requests.get(url+proj_unique_id,headers=key)
        if response.status_code == 200:
            return json.loads(response.text)
    elif type in ['project_type', 'specification', 'suitability']:
        response = requests.get(url + proj_unique_id + '/' + type, headers=key)
        if response.status_code == 200:
            return {'json': json.loads(response.text), 'status_code': response.status_code}
        else:
            dummy_json = {key: None for key in project_type_keys} if type == 'project_type' else ({key: None for key in specification_keys} if type == 'specification' else {key: None for key in suitability_keys})
            return {'json': dummy_json, 'status_code': response.status_code}

In [ ]:
scb_funds = pd.DataFrame(FundFactAPIGet('https://api.sec.or.th/FundFactsheet/fund/amc/','C0000000239','amc',FundFactKey))
scb_funds

In [ ]:
scb_funds = scb_funds.drop(columns=['last_upd_date'])
scb_funds = scb_funds[scb_funds['fund_status'] == 'RG']
scb_funds = scb_funds.reset_index(drop=True)

In [ ]:
scb_funds

In [ ]:
project_type_keys = ['proj_retail_type', 'proj_term_flag', 'proj_term_dd', 'proj_term_mm','proj_term_yy']
specification_keys = ['spec_code', 'spec_desc']
suitability_keys = ['risk_spectrum', 'risk_spectrum_desc','fund_suitable_desc']

In [ ]:
for index,row in scb_funds.iterrows():
    try:
        response = FundFactAPIGet('https://api.sec.or.th/FundFactsheet/fund/',str(row['proj_id']),'project_type',FundFactKey)
        scb_funds.at[index, 'proj_retail_type'] = response['json']['proj_retail_type']
        scb_funds.at[index, 'proj_term_flag'] = response['json']['proj_term_flag']
        scb_funds.at[index, 'proj_term_dd'] = response['json']['proj_term_dd']
        scb_funds.at[index, 'proj_term_mm'] = response['json']['proj_term_mm']
        scb_funds.at[index, 'proj_term_yy'] = response['json']['proj_term_yy']
    except Exception as e:
        pass
    try:
        response = FundFactAPIGet('https://api.sec.or.th/FundFactsheet/fund/',str(row['proj_id']),'specification',FundFactKey)
        scb_funds.at[index, 'spec_code'] =  ','.join(sub_res['spec_code'] for sub_res in response['json'])
        scb_funds.at[index, 'spec_desc'] =  ','.join(sub_res['spec_desc'] for sub_res in response['json']) 
    except Exception as e:
        pass
    try:
        response = FundFactAPIGet('https://api.sec.or.th/FundFactsheet/fund/',str(row['proj_id']),'suitability',FundFactKey)
        scb_funds.at[index, 'risk_spectrum'] = response['json']['risk_spectrum']
        try:
            scb_funds.at[index, 'risk_spectrum_desc'] = base64.b64decode(response['json']['risk_spectrum_desc']).decode('utf-8')
        except Exception as e:
            scb_funds.at[index, 'risk_spectrum_desc'] = None
        try:
            scb_funds.at[index, 'fund_suitable_desc'] = base64.b64decode(response['json']['fund_suitable_desc']).decode('utf-8')
        except Exception as e:
            scb_funds.at[index, 'fund_suitable_desc'] = None
    except Exception as e:
        pass
    time.sleep(0.1)

In [ ]:
scb_funds

In [ ]:
scb_funds.to_csv('../Result_amc/SCB/scb_funds.csv', encoding='utf8')

In [ ]:
scb_funds['spec_code'].value_counts()